<a href="https://colab.research.google.com/github/hhtnghia321/PPCA---project---Group-NNNV/blob/master/fillna_with_PPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
oridata_source = 'https://drive.google.com/u/1/uc?id=1iDCKJQbhoIp23YYZSmxRjub2b91TQmzJ&export=download'
dataNA_source = 'https://drive.google.com/u/1/uc?id=1cz7oJ0BJR5rzH1dVTtq2SJPdhxFEYs9o&export=download'
data_fillNA_source = 'https://drive.google.com/u/1/uc?id=1c6AAacF2piMdrvv6ZU5AwIHwXOo7xvJS&export=downloadd'
pos_of_true_values = 'https://drive.google.com/u/1/uc?id=1B7VOB_qM08uRHWLFvhyd9Hq95CN0OcMz&export=download'

In [0]:
import numpy as np
import random
import pandas as pd

In [0]:
names = ['Sample_code_number', 'Clump_Thickness', 'Uniformity_of_Cell_Size', 'Uniformity_of_Cell_Shape',
        'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
        'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class']
ori_data = pd.read_csv(oridata_source, names=names, index_col = False, na_values = '?')
ori_data.fillna(round(ori_data.mean()), inplace = True)
mean_ori_data = ori_data.mean()[1:-1]
ori_data = ori_data.drop(columns=['Sample_code_number', 'Class']).values

In [0]:
names_ = ['Index', 'Clump_Thickness', 'Uniformity_of_Cell_Size', 'Uniformity_of_Cell_Shape',
        'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
        'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses']
data = pd.read_csv(data_fillNA_source, index_col = False, names= names_)
data.shape
data = data.iloc[:,1:].values
print(data.shape)

In [0]:
def PPCA(data, q, sigma_init = 1, EM = False):
    N = data.shape[0]  # N row, N t_i
    d = data.shape[1]  # d cột của data
    mean = np.mean(data.T, axis=1) # mean vector
    mean = np.array([mean]).T # chuyển vecto mean về vecto cột
    S = (data.T -  mean).dot((data.T -  mean).T) / N # hiệp phương sai sample S
    if EM == False:
      eig_vals, eig_vecs = np.linalg.eig(S) # hàm riêng và trị riêng của S
      # xếp các trị riêng hàm riêng tương ứng thành các tuple
      eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
      # xếp lại theo thứ tự theo giá trị trị riêng
      eig_pairs.sort() 
      eig_pairs.reverse()
      # tìm MLE cho lambda
      lambda_ = 0
      for i in range(q, d):
        lambda_ += eig_pairs[i][0]
      sigma_ML = 1/(d-q)*lambda_
      # tìm ma trận U_q
      U_q = eig_pairs[0][1]
      for i in range(1, q):
        U_q = np.vstack((U_q, eig_pairs[i][1]))
      U_q = U_q.T
      # tìm ma trận Delta_q
      eig_vals_for_delta = np.array(eig_pairs[0][0])
      for i in range(1, q):
        eig_vals_for_delta = np.append(eig_vals_for_delta, eig_pairs[i][0])
      Delta_q = np.diag((eig_vals_for_delta))
      # tính ma trận W với giả định R = I
      W = np.matrix.round( U_q.dot(  np.sqrt(Delta_q - sigma_ML*np.identity(q)) ), 2 )
    else:
        sigma = sigma_init 
        W = np.random.rand(d, q)
        while True:
            M = W.T.dot(W) + sigma * np.identity(q)
            M_inv = np.linalg.inv(M)
    
            W_ml = S.dot(W).dot(np.linalg.inv(sigma*np.identity(q) + M_inv.dot(W.T).dot(S).dot(W)))
            sigma_ML = (1/d) * np.trace(S - S.dot(W).dot(M_inv).dot(W_ml.T))
            sigma_ML = np.absolute(sigma_ML)
            if abs((sigma - sigma_ML)) <= 0.00001:
                break
            W = W_ml
            sigma = sigma_ML
    return (W, sigma_ML, mean)

In [0]:
W_ML = PPCA(data, 2, EM=True)
W = W_ML[0]
t_n = (W.dot(np.linalg.inv(W.T.dot(W))).dot(W.T).dot(data.T)).T
#t_n = t_n.T
t_n.shape

In [0]:
ori_data

In [0]:
pos = pd.read_csv(pos_of_true_values, index_col=False)
pos = pos.iloc[:, 1:]
pos.iloc[:, 0]
result_ML = [] 
result_mean = []
for i in range(0, pos.shape[1]):
  error_ML = []
  error_mean = []
  for item in pos.iloc[:, i]:
    error_ML.append((ori_data[item,i] - t_n[item,i])**2)
    error_mean.append((ori_data[item,i] - mean_ori_data[i])**2)
  result_ML.append((sum(error_ML)/50)**0.5)
  result_mean.append((sum(error_mean)/50)**0.5)

print(result_ML)
print(result_mean)


##Compared Calculation between MLE (closed form) and EM algorithm for PCA


In [0]:
%time 
EM = PPCA(data, 3, EM=True)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [0]:
%time 
MLE = PPCA(data, 3)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.53 µs


##Compare PCA with PPCA (with missing value)
*in this section, we try to illustrate how good EM of PPCA to reconduct missing value

In [0]:
from sklearn.decomposition import  PCA
pca= PCA(2, random_state=2020)
pca.fit(data)
pca_comps = pca.components_
pca_data = pca_comps.dot(data.transpose()).transpose()
pca_data

array([[ 4.90678866,  0.27009041],
       [13.23332924, -5.29317358],
       [ 5.54514462, -0.06107697],
       ...,
       [16.84522538,  6.3071654 ],
       [14.59057013,  2.49647452],
       [16.42327784,  2.61090331]])

In [0]:
data_miss = pd.read_csv(dataNA_source,na_values="?")
data_miss.fillna(data_miss.mean()).iloc[:,1:].values
ppca_comp_miss = PPCA(data_miss, 2,EM=True)

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
